In [17]:
import os
import torch
import torchvision.transforms as transforms
from torchvision import models
from imblearn.under_sampling import NearMiss
import numpy as np
from sklearn.preprocessing import LabelEncoder
import shutil
from collections import Counter
from PIL import Image

In [38]:
dataset_path = '../../ORIGINAL DATASET'
balanced_dataset_path = '../../balanced_dataset'

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
resnet = models.resnet50(pretrained=True)
resnet.fc = torch.nn.Identity() 
resnet = resnet.to(device)
resnet.eval()

d:\workspace\ThesisProject\venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\workspace\ThesisProject\venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [5]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [12]:
def extract_features(image_path):
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0).to(device)  # Add batch dimension
    with torch.no_grad():
        features = resnet(image).cpu().numpy().flatten()  # Extract and flatten the features
    return features

# Prepare lists for features, labels, and image paths
features = []
labels = []
image_paths = []

In [13]:
for class_name in os.listdir(dataset_path):
    class_folder = os.path.join(dataset_path, class_name)
    if os.path.isdir(class_folder):
        for image_name in os.listdir(class_folder):
            image_path = os.path.join(class_folder, image_name)
            try:
                feature_vector = extract_features(image_path)
                features.append(feature_vector)
                labels.append(class_name)
                image_paths.append(image_path)
            except Exception as e:
                print(f"Error processing {image_path}: {e}")






In [14]:
# Convert labels to numeric format
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(labels)

In [25]:
import os
import shutil
import numpy as np
from PIL import Image
from torchvision import models, transforms
import torch
from imblearn.under_sampling import NearMiss
from sklearn.preprocessing import LabelEncoder
from collections import Counter

# Set up device and model for feature extraction
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
resnet = models.resnet50(pretrained=True)
resnet.fc = torch.nn.Identity() 
resnet = resnet.to(device)
resnet.eval()

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

def extract_features(image_path):
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0).to(device)  # Add batch dimension
    with torch.no_grad():
        features = resnet(image).cpu().numpy().flatten()  # Extract and flatten features
    return features

# Prepare lists for features, labels, and image paths
features = []
labels = []
image_paths = []

for class_name in os.listdir(dataset_path):
    class_folder = os.path.join(dataset_path, class_name)
    if os.path.isdir(class_folder):
        for image_name in os.listdir(class_folder):
            image_path = os.path.join(class_folder, image_name)
            try:
                feature_vector = extract_features(image_path)
                features.append(feature_vector)
                labels.append(class_name)
                image_paths.append(image_path)
            except Exception as e:
                print(f"Error processing {image_path}: {e}")

# Convert labels to numeric format
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(labels)




ValueError: The target 'y' needs to have more than 1 class. Got 1 class instead

Class distribution in y: Counter({np.int64(8): 12112, np.int64(2): 4049, np.int64(7): 3593, np.int64(16): 2683, np.int64(6): 2534, np.int64(12): 450, np.int64(14): 299, np.int64(9): 241, np.int64(1): 218, np.int64(11): 206, np.int64(5): 190, np.int64(17): 181, np.int64(4): 147, np.int64(0): 98, np.int64(22): 90, np.int64(18): 56, np.int64(19): 49, np.int64(10): 42, np.int64(3): 41, np.int64(15): 29, np.int64(20): 25, np.int64(21): 21, np.int64(13): 16})


In [37]:
from collections import Counter
import numpy as np
from imblearn.under_sampling import NearMiss
import os
import shutil

# Step 1: Check the class distribution
class_counts = Counter(y)
print("Class distribution in y:", class_counts)

# Identify the top N majority classes (e.g., top 5 based on count)
top_n = 5  # Define how many top majority classes you want
top_classes = [cls for cls, count in class_counts.most_common(top_n)]

# Identify minority classes
minority_classes = [cls for cls in class_counts if cls not in top_classes]

print("Top majority classes:", top_classes)
print("Minority classes:", minority_classes)

# Step 2: Separate the majority and minority class samples
majority_class_indices = [i for i, label in enumerate(y) if label in top_classes]
minority_class_indices = [i for i, label in enumerate(y) if label in minority_classes]

print(f"Number of majority class samples: {len(majority_class_indices)}")
print(f"Number of minority class samples: {len(minority_class_indices)}")

X_majority = np.array([features[i] for i in majority_class_indices])
y_majority = np.array([y[i] for i in majority_class_indices])

X_minorities = np.array([features[i] for i in minority_class_indices])
y_minorities = np.array([y[i] for i in minority_class_indices])

# Debug: Check unique values in y_majority before resampling
print("Unique classes in y_majority before resampling:", np.unique(y_majority))

# Step 3: Apply NearMiss to the majority classes (top N)
near_miss = NearMiss(version=1)

# Resampling the majority classes
X_majority_resampled, y_majority_resampled = near_miss.fit_resample(X_majority, y_majority)

# Debug: Check the result after resampling
print(f"Resampled majority class size: {len(X_majority_resampled)}")
print("Unique classes in resampled y_majority:", np.unique(y_majority_resampled))

# Step 4: Combine the resampled majority class with the unchanged minority classes
X_resampled = np.concatenate([X_majority_resampled, X_minorities], axis=0)
y_resampled = np.concatenate([y_majority_resampled, y_minorities], axis=0)

# Step 5: Get the indices of the resampled majority class
resampled_majority_indices = near_miss.sample_indices_

# Combine the resampled majority indices with the untouched minority indices
resampled_indices = np.concatenate([resampled_majority_indices, minority_class_indices])
image_paths_resampled = [image_paths[i] for i in resampled_indices]

# Step 6: Create the balanced dataset and copy images
balanced_dataset_path = 'path_to_balanced_dataset'  # Replace with actual path

if not os.path.exists(balanced_dataset_path):
    os.makedirs(balanced_dataset_path)

for image_path, class_index in zip(image_paths_resampled, y_resampled):
    class_name = label_encoder.inverse_transform([class_index])[0]
    class_folder = os.path.join(balanced_dataset_path, class_name)
    if not os.path.exists(class_folder):
        os.makedirs(class_folder)
    shutil.copy(image_path, class_folder)

print("Balanced dataset created at:", balanced_dataset_path)


Class distribution in y: Counter({np.int64(8): 12112, np.int64(2): 4049, np.int64(7): 3593, np.int64(16): 2683, np.int64(6): 2534, np.int64(12): 450, np.int64(14): 299, np.int64(9): 241, np.int64(1): 218, np.int64(11): 206, np.int64(5): 190, np.int64(17): 181, np.int64(4): 147, np.int64(0): 98, np.int64(22): 90, np.int64(18): 56, np.int64(19): 49, np.int64(10): 42, np.int64(3): 41, np.int64(15): 29, np.int64(20): 25, np.int64(21): 21, np.int64(13): 16})
Top majority classes: [np.int64(8), np.int64(2), np.int64(7), np.int64(16), np.int64(6)]
Minority classes: [np.int64(0), np.int64(1), np.int64(3), np.int64(4), np.int64(5), np.int64(9), np.int64(10), np.int64(11), np.int64(12), np.int64(13), np.int64(14), np.int64(15), np.int64(17), np.int64(18), np.int64(19), np.int64(20), np.int64(21), np.int64(22)]
Number of majority class samples: 24971
Number of minority class samples: 2399
Unique classes in y_majority before resampling: [ 2  6  7  8 16]
Resampled majority class size: 12670
Unique 